In [56]:
import pandas as pd
import numpy as np

%matplotlib inline
from tensorflow import keras

In [57]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/refs/heads/master/6_gradient_descent/insurance_data.csv"
)

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df[["age", "affordibility"]], df.bought_insurance, test_size=0.2, random_state=25
)

In [59]:
X_Train_scaled = X_train.copy()
X_Test_scaled = X_test.copy()

X_Test_scaled["age"] = X_Test_scaled["age"] / 100
X_Train_scaled["age"] = X_Train_scaled["age"] / 100

In [60]:
X_Train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [61]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        keras.layers.Dense(
            1, activation="sigmoid", kernel_initializer="ones", bias_initializer="zeros"
        ),
    ]
)

/Users/vanshchopra/Documents/playground/vc-playground/nn/1/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [64]:
coef, intercept = model.get_weights()
print("Coefficients:", coef)
print("Intercept:", intercept)

Coefficients: [[5.0608625]
 [1.408652 ]]
Intercept: [-2.9137027]


In [65]:
model.predict(X_Test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[0.70548475],
       [0.35569575],
       [0.16827832],
       [0.4780119 ],
       [0.72606957],
       [0.82949823]], dtype=float32)

In [67]:
def sigmoid(x):
    import numpy as np

    return 1 / (1 + np.exp(-x))


sigmoid(18)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.3550
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.3550


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.3550
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.3550


[0.35497769713401794, 1.0]

In [68]:
def prediction_function(age, affordability):
    weighted_sum = coef[0] * age + coef[1] * affordability + intercept
    return sigmoid(weighted_sum)

np.float64(0.9999999847700205)

In [69]:
prediction_function(0.47, 1)

array([0.33625335], dtype=float32)

In [73]:
prediction_function(0.18, 1)

array([0.3556957], dtype=float32)

In [71]:
def log_loss(y_true, y_pred):
    import numpy as np

    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss.mean()

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [85]:
def gradient_descent(age, affordability, y_true, epochs, loss_threshold):
    w1 = w2 = 1.0
    b = 0.0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + b
        y_pred = sigmoid(weighted_sum)
        loss = log_loss(y_true, y_pred)
        w1_grad = (1 / n) * np.dot(np.transpose(age), (y_pred - y_true))
        w2_grad = (1 / n) * np.dot(np.transpose(affordability), (y_pred - y_true))
        b_grad = (1 / n) * np.sum(y_pred - y_true)
        w1 -= rate * w1_grad
        w2 -= rate * w2_grad
        b -= rate * b_grad
        print(f"Epoch {i}, Loss: {loss}, w1: {w1}, w2: {w2}, b: {b}")
        if loss < loss_threshold:
            break
    return w1, w2, b

In [87]:
gradient_descent(
    X_Train_scaled["age"],
    X_Train_scaled["affordibility"],
    y_train,
    epochs=1000,
    loss_threshold=0.4631,
)

Epoch 0, Loss: 0.7113403233723417, w1: 0.974907633470177, w2: 0.948348125394529, b: -0.11341867736368583
Epoch 1, Loss: 0.681264778737757, w1: 0.9556229728273669, w2: 0.9058873696677865, b: -0.2122349122718517
Epoch 2, Loss: 0.6591474252715025, w1: 0.9416488476693794, w2: 0.8719790823960313, b: -0.2977578997796538
Epoch 3, Loss: 0.6431523291301917, w1: 0.9323916996249162, w2: 0.8457541517722915, b: -0.3715094724003511
Epoch 4, Loss: 0.6316873063379158, w1: 0.9272267472726993, w2: 0.8262362885332687, b: -0.43506643026891584
Epoch 5, Loss: 0.623471707997592, w1: 0.9255469396815343, w2: 0.8124402814952774, b: -0.48994490058938817
Epoch 6, Loss: 0.6175321183044205, w1: 0.9267936114129968, w2: 0.8034375029757677, b: -0.5375299543522853
Epoch 7, Loss: 0.6131591858705934, w1: 0.93047170420295, w2: 0.7983920007454487, b: -0.5790424270894963
Epoch 8, Loss: 0.6098518179750948, w1: 0.9361540784567942, w2: 0.7965748796787705, b: -0.6155315088627655
Epoch 9, Loss: 0.6072639970231438, w1: 0.94347912

(np.float64(5.051047623653049),
 np.float64(1.4569794548473887),
 np.float64(-2.9596534546250037))

In [91]:
class myNN:
    def __init__(self):
        self.w1 = 1.0
        self.w2 = 1.0
        self.b = 0.0

    def fit(self, X, y, epochs, loss_threshold):
        age = X["age"]
        affordability = X["affordibility"]
        return self.gradient_descent(age, affordability, y, epochs, loss_threshold)

    def predict(self, X):
        age = X["age"]
        affordability = X["affordibility"]
        weighted_sum = self.w1 * age + self.w2 * affordability + self.b
        return sigmoid(weighted_sum)

    def gradient_descent(self, age, affordability, y_true, epochs, loss_threshold):
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = self.w1 * age + self.w2 * affordability + self.b
            y_pred = sigmoid(weighted_sum)
            loss = log_loss(y_true, y_pred)
            w1_grad = (1 / n) * np.dot(np.transpose(age), (y_pred - y_true))
            w2_grad = (1 / n) * np.dot(np.transpose(affordability), (y_pred - y_true))
            b_grad = (1 / n) * np.sum(y_pred - y_true)
            self.w1 -= rate * w1_grad
            self.w2 -= rate * w2_grad
            self.b -= rate * b_grad
            print(f"Epoch {i}, Loss: {loss}, w1: {self.w1}, w2: {self.w2}, b: {self.b}")
            if loss < loss_threshold:
                break
        return self.w1, self.w2, self.b

In [93]:
customModel = myNN()
customModel.fit(X_Train_scaled, y_train, epochs=5000, loss_threshold=0.4631)

Epoch 0, Loss: 0.7113403233723417, w1: 0.974907633470177, w2: 0.948348125394529, b: -0.11341867736368583
Epoch 1, Loss: 0.681264778737757, w1: 0.9556229728273669, w2: 0.9058873696677865, b: -0.2122349122718517
Epoch 2, Loss: 0.6591474252715025, w1: 0.9416488476693794, w2: 0.8719790823960313, b: -0.2977578997796538
Epoch 3, Loss: 0.6431523291301917, w1: 0.9323916996249162, w2: 0.8457541517722915, b: -0.3715094724003511
Epoch 4, Loss: 0.6316873063379158, w1: 0.9272267472726993, w2: 0.8262362885332687, b: -0.43506643026891584
Epoch 5, Loss: 0.623471707997592, w1: 0.9255469396815343, w2: 0.8124402814952774, b: -0.48994490058938817
Epoch 6, Loss: 0.6175321183044205, w1: 0.9267936114129968, w2: 0.8034375029757677, b: -0.5375299543522853
Epoch 7, Loss: 0.6131591858705934, w1: 0.93047170420295, w2: 0.7983920007454487, b: -0.5790424270894963
Epoch 8, Loss: 0.6098518179750948, w1: 0.9361540784567942, w2: 0.7965748796787705, b: -0.6155315088627655
Epoch 9, Loss: 0.6072639970231438, w1: 0.94347912

Epoch 0, Loss: 0.7113403233723417, w1: 0.974907633470177, w2: 0.948348125394529, b: -0.11341867736368583
Epoch 1, Loss: 0.681264778737757, w1: 0.9556229728273669, w2: 0.9058873696677865, b: -0.2122349122718517
Epoch 2, Loss: 0.6591474252715025, w1: 0.9416488476693794, w2: 0.8719790823960313, b: -0.2977578997796538
Epoch 3, Loss: 0.6431523291301917, w1: 0.9323916996249162, w2: 0.8457541517722915, b: -0.3715094724003511
Epoch 4, Loss: 0.6316873063379158, w1: 0.9272267472726993, w2: 0.8262362885332687, b: -0.43506643026891584
Epoch 5, Loss: 0.623471707997592, w1: 0.9255469396815343, w2: 0.8124402814952774, b: -0.48994490058938817
Epoch 6, Loss: 0.6175321183044205, w1: 0.9267936114129968, w2: 0.8034375029757677, b: -0.5375299543522853
Epoch 7, Loss: 0.6131591858705934, w1: 0.93047170420295, w2: 0.7983920007454487, b: -0.5790424270894963
Epoch 8, Loss: 0.6098518179750948, w1: 0.9361540784567942, w2: 0.7965748796787705, b: -0.6155315088627655
Epoch 9, Loss: 0.6072639970231438, w1: 0.94347912

(np.float64(5.051047623653049),
 np.float64(1.4569794548473887),
 np.float64(-2.9596534546250037))

In [94]:
customModel.predict(X_Test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

In [38]:
y_test

7     1
21    0
5     1
2     1
13    0
19    0
Name: bought_insurance, dtype: int64